# Imports and preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset

In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.0.1+cu117
CUDA version: 11.7
cuDNN version: 8500
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.32.1
Datasets version: 2.14.4


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Tue Dec 12 08:12:37 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-PCIE-40GB           On | 00000000:01:00.0 Off |                    0 |
| N/A   27C    P0               35W / 250W|    980MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |
|  1 |  0% |  0% |
|  2 |  0% |  5% |
|  3 |  0% | 37% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |
|  1 |  7% |  1% |
|  2 |  0% |  5% |
|  3 |  0% | 37% |


In [8]:
# Smaller and faster than bert.
model_ckpt = "distilbert-base-uncased"

epochs = 3 #Number of full cycles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evaluation examples in on iteration.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

Load dataset from huggingface

In [9]:
dataset = load_dataset("kristmh/low_vs_not_priority")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/24705 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/197631 [00:00<?, ? examples/s]

Generating validate split:   0%|          | 0/24703 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__'],
        num_rows: 24705
    })
    train: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__'],
        num_rows: 197631
    })
    validate: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__'],
        num_rows: 24703
    })
})

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

## Tokenization

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [14]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

In [15]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/24705 [00:00<?, ? examples/s]

Map:   0%|          | 0/197631 [00:00<?, ? examples/s]

Map:   0%|          | 0/24703 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 24705
    })
    train: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 197631
    })
    validate: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 24703
    })
})

In [16]:
# List the columns of the dataset.
# Should be: ["attention_mask", "input_ids", "labels", "token_type_ids"]
# Remove unnecessary columns that the model does not expect.
tokenized_dataset = tokenized_dataset.remove_columns(["text_clean", "class", "__index_level_0__"])

train_dataset = tokenized_dataset["train"]
print(train_dataset)
validation_dataset = tokenized_dataset["validate"]
print(validation_dataset)
test_dataset = tokenized_dataset["test"]
test_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 197631
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 24703
})


Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 24705
})

## Training a classifier

In [17]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

In [19]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.573600,0.617348,0.677205,[0.7165707 0.62514103],[0.65835021 0.70794293],[0.7860875 0.55968013]
2,0.490900,0.568892,0.701008,[0.74162177 0.64524496],[0.67246083 0.75134228],[0.82663963 0.56540404]
3,0.438800,0.603542,0.704692,[0.72416531 0.68225968],[0.70287728 0.70692301],[0.74678312 0.65925926]


TrainOutput(global_step=37056, training_loss=0.5127260469813832, metrics={'train_runtime': 9854.6135, 'train_samples_per_second': 60.164, 'train_steps_per_second': 3.76, 'total_flos': 7.85389933913518e+16, 'train_loss': 0.5127260469813832, 'epoch': 3.0})

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, validation loss increases = Overfitting

In [20]:
# Evaluate validation set
eval_result = trainer.evaluate(eval_dataset=validation_dataset)

In [21]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 3.0

eval_accuracy = 0.7046917378456058

eval_f1 = [0.72416531 0.68225968]

eval_loss = 0.6035424470901489

eval_precision = [0.70287728 0.70692301]

eval_recall = [0.74678312 0.65925926]

eval_runtime = 327.1803

eval_samples_per_second = 75.503

eval_steps_per_second = 2.36



In [22]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [23]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 3.0

eval_accuracy = 0.7086419753086419

eval_f1 = [0.72415115 0.69128495]

eval_loss = 0.5932432413101196

eval_precision = [0.70618133 0.71154865]

eval_recall = [0.74305938 0.67214345]

eval_runtime = 262.5138

eval_samples_per_second = 94.109

eval_steps_per_second = 2.945



In [24]:
trainer.save_model(model_dir + "_local") 

In [25]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [26]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [27]:
classifier("this does not need to be done fast")

[{'label': 'LABEL_1', 'score': 0.5804873108863831}]

In [28]:
classifier("this is super important")

[{'label': 'LABEL_0', 'score': 0.5831249952316284}]

In [29]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_1', 'score': 0.5824448466300964}]

## Important to delete large objects to free memory 
del train_dataset

In [30]:
del validation_dataset 

In [31]:
del model

In [32]:
# Free cache
torch.cuda.empty_cache()

In [33]:
!nvidia-smi

Tue Dec 12 11:10:11 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-PCIE-40GB           On | 00000000:01:00.0 Off |                    0 |
| N/A   32C    P0               36W / 250W|   2610MiB / 40960MiB |      2%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--